# Libraries 

In [1]:
# Standard Libraries
import os, sys
import pandas as pd
import numpy as np
import datetime
import random

# For Notifications
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

# For plotting and charts


# An attempt to reach out to sharepoint site and pull data
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.file import File

# Visualiztion Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

Table of Contents
-----------------

1. [File Imports](#File-Imports)
2. [Sample Demographics](#Sample-Demographics)
3. [Pricing Analysis](#Pricing-Analysis)
4. [Software Decision Making](#Software-Decision-Making)
5. [Products](#Products)
6. [Deployment Method](#Deployment-Method)
7. [Payment Method](#Payment-Method)

## File Imports

For ease of access:
    
    Location: /../Downloads/
    Filename: temp-survey data.xlsx 
      Sheets: RAW DATA, Dictionary Mapping

In [2]:
current_dir = os.getcwd()
source = "C:/Users/luiz.frias/Documents/Applications/Python Scripts/CaPDB/Templates"
os.chdir(source) # Setting Source to look from top level folder

# This is to call in functions for data handling
import FileImports as fimp

os.chdir(current_dir)

In [3]:
# Importing data from an excel workbook
t = fimp.multisheets(2,f_name='temp-survey data.xlsx',sheets=['RAW DATA','Dictionary Mapping'])

In [4]:
# Setting DF with all values, and question mapping
wkgdf = t[list(t.keys())[0]]
dmapper_sheet = t[list(t.keys())[1]]

In [5]:
# Return questions that are in the Qualtrics Survey
mdict = dmapper_sheet.dropna(subset=['Category']).copy()

In [6]:
# Clean up module language
mdict.loc[:, 'Modules'] = mdict.loc[:, 'Modules'].replace(' -','',regex=True).replace(' / ','/',regex=True)
CST_modules = mdict.Modules.dropna().unique().tolist()

In [7]:
# Create Module list by Public Sector
modules_by_sector = dict({
    'PSJ': CST_modules[:6],
    'PA': CST_modules[-4:]
})

## Sample Demographics

Sample Demographics Quicklinks:
1. [PSJ v. PSJ](#Public-Sector)
2. [Population](#Population-Served)
3. [Sworn Officers](#Sworn-Officers)
4. [Is decision maker](#Decision-Makers)
5. [Modules-PA](#Modules---PA)
6. [Modules-PSJ](#Modules---PSJ)

Go to [Next](#Pricing-Analysis) or [Previous](#Table-of-Contents)

Return to [Top](#Libraries)

In [8]:
def genericBar_Plot(*arg,title=None, x=None, y=None, N=None):
    """
    Created to plot a generic Bar Plot. 
    
    Parameters
    ----------
    *arg : pd.Series
        This works with a value_count() series with:
            x = pd.Series.Index
            y = pd.Series.values
    
    Return
    ------
    Plot.ly Dashboard
    
    Examples
    --------
    Product_line_Revenue.value_counts()
    """
    traces = []
    for arg in arg:
        traces.append(
            go.Bar(
                x=arg.index,
                y=arg.values/arg.sum(),
                text=np.round(arg.values/arg.sum(),2),
                textposition='auto'
            )
        )
    data = traces
    layout = go.Layout(
        title=title,
        xaxis=dict(
            title=arg.name        
        ),
        yaxis=dict(
            title='% of {} Respondents'.format(arg.name)
        ),
    )
    
    plotly.offline.iplot({
        'data':data, 
        'layout':layout}
    )    
    Num = N if N!=None else arg.sum()
    print("N = {}".format(Num))

In [9]:
def grouped_Bar_Chart(data=None, title=None):
    """
    Created to plot a generic Bar Plot. 
    
    Parameters
    ----------
    data : pd.DataFrame
        This is to create multiple grouped bars based on 'Yes'/'No' :
            x = pd.DataFrame.column.value_counts.Index
            y = pd.Series.values
    
    Return
    ------
    Plot.ly Dashboard
    
    Examples
    --------
    Product_line_Revenue.value_counts()
    """
    
    data_cols = []
    
    for col in data.columns:
        data_cols.append(data[col].value_counts())
       
    traces = []
    
    for arg in data_cols:
        traces.append(
            go.Bar(
                y=arg.values/arg.sum(),
                x=arg.index,
                name=arg.name,
                text=np.round(arg.values/arg.sum(),2),
                textposition='auto'
#                 orientation='h'
            )       
        )
    
    layout = go.Layout(
        title=title,
        xaxis=dict(
            title='Module Ownership'
        ),
        yaxis=dict(
            title='% of Respondents'
        ),
        barmode='group'
    )
    
    plotly.offline.iplot({
        'data':traces,
        'layout':layout
    })
    print("N = {}".format(len(data)))

In [10]:
def generic_heatmap(*arg,title=None, N=None):
    """
    Created to plot a generic heatmap. 
    
    Parameters
    ----------
    *arg : 2D array with the categorical index & columns
        This will most likely be a pivot table
            x = pd.MultiIndex columns [y for x,y in arg.columns.tolist()]
            y = pd.DataFrame.Index
            z = pd.DataFrame.values
    
    Return
    ------
    Plot.ly Dashboard
    
    Examples
    --------
    Product_line_Revenue.pivot
    """
    traces = []
    for arg in arg:
        traces.append(
            go.Heatmap(
                x=[b for a,b in arg.columns.tolist()],
                y=arg.index,
                z=arg.values,
                text=np.round(arg.values/arg.sum().values,2),
#                 textposition='auto'
            )
        )
    data = traces
    layout = go.Layout(
        title=title,
        xaxis=dict(
            title='Weekday'        
        ),
        yaxis=dict(
            title='Time of Day'
        ),
    )
    
    plotly.offline.iplot({
        'data':data, 
        'layout':layout}
    )    
    Num = N if N!=None else arg.sum()
    print("N = {}".format(Num))

### Public Sector

In [11]:
genericBar_Plot(wkgdf['Public Sector'].value_counts(),
                title='Respondents')

N = 415


### Population Served

In [12]:
genericBar_Plot(wkgdf['Population Band'].value_counts(),
                title='Respondents')

N = 415


### Sworn Officers

In [13]:
genericBar_Plot(wkgdf.where(~wkgdf.isin(['---']),np.nan)['Sworn Officer Band'].value_counts(),
                title='Respondents')

N = 309


### Decision Makers

In [14]:
genericBar_Plot(wkgdf['Decision Levels'].value_counts(),
                title='Respondents')

N = 415


### Modules - PA

In [15]:
# Need to get slice of df to show only PA (100+ Respondents) and what modules are owned from each
PA_only_indicies = wkgdf[wkgdf['Public Sector']=='PA'].index

# Obtain the Question Headers mapped from mdict
PA_mdict_module_ownership_mask = mdict.Category == 'Module Ownership'
PA_mdict_mod_ownshp_table = mdict[PA_mdict_module_ownership_mask] #Isolates table for Mod Ownership
PA_Mod_ownshp_questions = PA_mdict_mod_ownshp_table[PA_mdict_mod_ownshp_table['Modules']\
                                                           .isin(modules_by_sector['PA'])
                                                          ]['Question #'].tolist()
# Add in the Qualtrics Prefix to map the correct header
PA_Mod_ownshp_headers = ['Qualtrics '+x for x in PA_Mod_ownshp_questions]

#Create PA Module Ownership Table
PA_mod_ownshp_count_df = pd.DataFrame(wkgdf.loc[PA_only_indicies, 
                                                pd.Index(PA_Mod_ownshp_headers)].values,
                                      index=PA_only_indicies,
                                      columns=modules_by_sector['PA'])

In [16]:
grouped_Bar_Chart(data=PA_mod_ownshp_count_df,
                  title='PA Respondents')

N = 106


### Modules - PSJ

In [17]:
# Need to get slice of df to show only PA (100+ Respondents) and what modules are owned from each
PSJ_only_indicies = wkgdf[wkgdf['Public Sector']=='PSJ'].index

# Obtain the Question Headers mapped from mdict
PSJ_mdict_module_ownership_mask = mdict.Category == 'Module Ownership'
PSJ_mdict_mod_ownshp_table = mdict[PSJ_mdict_module_ownership_mask] #Isolates table for Mod Ownership
PSJ_Mod_ownshp_questions = PSJ_mdict_mod_ownshp_table[PSJ_mdict_mod_ownshp_table['Modules']\
                                                           .isin(modules_by_sector['PSJ'])
                                                          ]['Question #'].tolist()
# Add in the Qualtrics Prefix to map the correct header
PSJ_Mod_ownshp_headers = ['Qualtrics '+x for x in PSJ_Mod_ownshp_questions]

# Create PSJ Non-Full Suite indicies
PSJ_non_full_suite_mask = wkgdf.loc[PSJ_only_indicies,PSJ_Mod_ownshp_headers[-1:][0]].isin(['No'])
PSJ_non_full_suite_indicies = wkgdf.loc[PSJ_only_indicies,
                                        PSJ_Mod_ownshp_headers[-1:][0]
                                       ][PSJ_non_full_suite_mask].index

# Add to dataframe for easier filtering
wkgdf['PSJ-FULLSUITE OWN'] = np.where(wkgdf.loc[:,PSJ_Mod_ownshp_headers[-1:][0]].isin(['No']),0.0,1.0)

# Create PA Module Ownership Table
PSJ_mod_ownshp_count_df = pd.DataFrame(wkgdf.loc[PSJ_non_full_suite_indicies, 
                                                pd.Index(PSJ_Mod_ownshp_headers[:-1])].values,
                                      index=PSJ_non_full_suite_indicies,
                                      columns=modules_by_sector['PSJ'][:-1])

In [18]:
grouped_Bar_Chart(data=PSJ_mod_ownshp_count_df,
                  title='PSJ Respondents - Non-Full Suite')

N = 220


## Pricing Analysis

Pricing Analysis Quicklinks:
1. [Van Westendorp](#Van-Westendorp)
2. [Gabor Granger](#Gabor-Granger)
3. [Charge Type Distribution](#Charge-Type-Distribution)

Go to [Next](#Software-Decision-Making) or [Previous](#Sample-Demographics)

Return to [Top](#Libraries)

### Van Westendorp

### Gabor Granger

In [19]:
def yes_to_val(x):
    '''
    Takes yes values and converts them to $ values based on their start price.
    Made for Gabor Granger Analysis
    DF format should be:
    Key | 'x' Start Price | 1.0 | 1.2 | ... | 1.5 |
    '''
    headers = ['Start Price',1.0, 1.1,1.2,1.3,1.4,1.5]
    # Splitting the df first
    tmpdf_pt1 = x.iloc[:, :2].copy()
    tmpdf_pt2 = x.iloc[:, 2:].copy()
    
    # Obtaining the columns in pt2 to loop through
    df_col_parser = tmpdf_pt2.columns.copy()
    
    # Getting the module start price string to use for index slicing - to set values on split data frame pt1
    # idx | pt2 <- apply formula to pt1 and save over in pt2 (replacing y/n with values)
    start_price_col = tmpdf_pt1.columns[tmpdf_pt1.columns.str.contains('Start Price')][0]
    
    # Applying pricing to relevant columns (Yes only)
    for i in range(len(df_col_parser)):
        mask = tmpdf_pt2[df_col_parser[i]].isin(['Yes'])
        tmpdf_pt2.loc[mask,df_col_parser[i]] = tmpdf_pt1.loc[:,start_price_col][mask] * headers[-6:][i]
        tmpdf_pt2.loc[~mask,df_col_parser[i]] = 0.0
    
    tmpdf_pt2 = tmpdf_pt2.astype('float')
    tmpdf = tmpdf_pt1.join(tmpdf_pt2)
    
    return tmpdf

In [20]:
# Creating a holder for all Gabor Granger dfs
GG = {}
sfx = ['Start Price',1.0, 1.1,1.2,1.3,1.4,1.5]
for i in range(len(CST_modules)):
    cols = [CST_modules[i]+" "+str(x) for x in sfx]
    questions = mdict['Question #'].where((mdict['Methodology']=='Gabor Granger')&
                                          (mdict['Modules']==CST_modules[i])|
                          (mdict['Pricing Subset']=='Start Price')&
                                          (mdict['Modules']==CST_modules[i])
                         ).dropna().tolist()
    questions = ['Qualtrics '+x for x in questions]
    questions.append('Key')
    cols.append('Key')
    tmpdf = pd.DataFrame(wkgdf.loc[:,questions].copy())
    tmpdf.columns = cols.copy()
    
    # Reordering to have keys in front
    cols = tmpdf.columns.tolist()
    cols = cols[-1:]+cols[:-1]
    tmpdf = tmpdf[cols]
    
    # Cleanup and Presorting Values
    tmpdf = tmpdf.replace(to_replace='---',value=0)
    tmpdf = tmpdf[tmpdf[cols[2]]!='Does not appear in Qualtrics']
    # Function to apply values to y/n points
    tmpdf = yes_to_val(tmpdf)
    # Sort by Start Price Column
    tmpdf = tmpdf.sort_values(by=tmpdf.select_dtypes('float64').columns[0],ascending=False)
    
    GG.update({CST_modules[i]:tmpdf}) 

In [58]:
%%time
# '''
# This entire cell is meant to create a frequency table to measure what customer would pay at 
# x price point.
# Input = dictionary that carries all modules translated into numeric values from the Gabor Granger
# portion of a survey
# '''

# Creating a Frequency table to count the number of values that occur for each dataframe
GG_Fx = pd.DataFrame(np.arange(0,1500100,100), columns=['Frequency'])

# print("pt1")
# Inputing the frequncy count for each module
for k in GG:
#     print(k)
    freq_counts = []
    
    for i in range(len(GG_Fx.Frequency)):
        freq_counts.append(np.where(GG[k].iloc[:,2:]>GG_Fx['Frequency'].values[i],1.0,0.0).sum())
    
    GG_Fx[k] = freq_counts
    
    # Turn to percentage
    total_count = GG_Fx[k].nlargest(n=1).values
    GG_Fx[k] = GG_Fx[k]/total_count

# print("pt2")

# Creating a table to input Expected Revenue per one Unit sale @ x price (% x $)
GG_ExpRev = GG_Fx.copy()

# print("pt3")

# Applying the calculation per module
for item in GG_Fx.columns[1:]:
    GG_ExpRev[item] = GG_Fx['Frequency']*GG_Fx[item]

# print("pt4")

# Creating final table for charting purposes
GG_Final = GG_Fx.join(GG_ExpRev.drop(columns='Frequency'), rsuffix='_REV')

Wall time: 1min 19s


The end result table (GG_Final) should look a little something like this:

|Index|Frequency|CAD|911|RMS/FBR|Mobile|JMS|Full Suite|Finance|Community Development|Utilities|Asset/WorkManagement|CAD_REV|911_REV|RMS/FBR_REV|Mobile_REV|JMS_REV|Full Suite_REV|Finance_REV|Community Development_REV|Utilities_REV|Asset/Work Management_REV|
|---|
|0|0|1.0|1.0|1.0|1.0|1.0|1.0|1.0|1.0|1.0|1.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|
|1|100|0.8617283950617284|0.8071065989847716|0.8994082840236687|0.8605341246290801|0.8333333333333334|0.7976190476190477|0.8653846153846154|0.6707317073170732|0.8449612403100775|0.775|86.17283950617283|80.71065989847716|89.94082840236686|86.05341246290801|83.33333333333334|79.76190476190477|86.53846153846155|67.07317073170732|84.49612403100775|77.5|
|2|200|0.8592592592592593|0.8020304568527918|0.8964497041420119|0.8605341246290801|0.8253968253968254|0.7976190476190477|0.8269230769230769|0.6463414634146342|0.8294573643410853|0.75|171.85185185185185|160.40609137055836|179.28994082840237|172.10682492581603|165.07936507936506|159.52380952380955|165.3846153846154|129.26829268292684|165.89147286821705|150.0|
|3|300|0.8592592592592593|0.8020304568527918|0.8964497041420119|0.8605341246290801|0.8253968253968254|0.7976190476190477|0.8269230769230769|0.6463414634146342|0.8294573643410853|0.75|257.77777777777777|240.60913705583755|268.93491124260356|258.160237388724|247.6190476190476|239.2857142857143|248.07692307692307|193.90243902439025|248.8372093023256|225.0|
|4|400|0.8592592592592593|0.8020304568527918|0.8964497041420119|0.8605341246290801|0.8253968253968254|0.7976190476190477|0.8269230769230769|0.6463414634146342|0.8294573643410853|0.75|343.7037037037037|320.8121827411167|358.57988165680473|344.21364985163206|330.1587301587301|319.0476190476191|330.7692307692308|258.5365853658537|331.7829457364341|300.0|
|5|500|0.8567901234567902|0.7969543147208121|0.893491124260355|0.857566765578635|0.8174603174603174|0.7976190476190477|0.8269230769230769|0.6463414634146342|0.8294573643410853|0.75|428.3950617283951|398.47715736040607|446.7455621301775|428.7833827893175|408.73015873015873|398.80952380952385|413.46153846153845|323.1707317073171|414.72868217054264|375.0|
|6|600|0.8518518518518519|0.7868020304568528|0.8875739644970414|0.8516320474777448|0.8015873015873016|0.7976190476190477|0.8269230769230769|0.6463414634146342|0.8294573643410853|0.75|511.11111111111114|472.0812182741117|532.5443786982248|510.9792284866469|480.95238095238096|478.5714285714286|496.15384615384613|387.8048780487805|497.6744186046512|450.0|
|7|700|0.8518518518518519|0.7766497461928934|0.8875739644970414|0.8516320474777448|0.8015873015873016|0.7976190476190477|0.8269230769230769|0.6463414634146342|0.8294573643410853|0.75|596.2962962962963|543.6548223350254|621.301775147929|596.1424332344213|561.1111111111111|558.3333333333334|578.8461538461538|452.4390243902439|580.6201550387597|525.0|
|8|800|0.8518518518518519|0.7715736040609137|0.8875739644970414|0.8516320474777448|0.8015873015873016|0.7976190476190477|0.8269230769230769|0.6463414634146342|0.8294573643410853|0.75|681.4814814814815|617.258883248731|710.0591715976332|681.3056379821958|641.2698412698413|638.0952380952382|661.5384615384615|517.0731707317074|663.5658914728682|600.0|
|9|900|0.8518518518518519|0.7715736040609137|0.8875739644970414|0.8516320474777448|0.8015873015873016|0.7976190476190477|0.8269230769230769|0.6463414634146342|0.8294573643410853|0.75|766.6666666666666|694.4162436548223|798.8165680473372|766.4688427299703|721.4285714285714|717.8571428571429|744.2307692307692|581.7073170731708|746.5116279069767|675.0|
|10|1000|0.8493827160493828|0.7614213197969543|0.8816568047337278|0.8427299703264095|0.8015873015873016|0.7976190476190477|0.8269230769230769|0.6463414634146342|0.8294573643410853|0.75|849.3827160493828|761.4213197969543|881.6568047337278|842.7299703264096|801.5873015873017|797.6190476190477|826.9230769230769|646.3414634146342|829.4573643410853|750.0|

In [69]:
def interactive_GG(*df,title=None):
    '''
    Created to loop through or select chart data by module. 
    
    Parameters
    ----------
    *df : DataFrame
        Table
    
    Return
    ------
    Plot.ly Dashboard
    
    Examples
    --------
    DataFrame must look like
    |Index|Frequency|CAD|RMS/FBR|Finance|CAD_REV|RMS/FBR_REV|Finance_REV|
    |---|
    |0|0|1.0|1.0|1.0|0.0|0.0|0.0|
    |1|100|0.8617283950617284|0.8994082840236687|0.8653846153846154|86.17283950617283|89.94082840236686|86.53846153846155|
    '''
        
    Mod_dict = {}
    
    Modules = [x for x in df[0].columns if '_' not in x and 'Frequency' not in x]
    
    for df_arg in df:    
        # Loop through to create traces necessary to select GG plot
        for i in range(len(Modules)):
            # Creating an empty list that carries 2 traces for each Module iterated through
            traces = []
            
            # Module Frequency Values
            traces.append(
                go.Scatter(
                    x = df_arg.loc[:, 'Frequency'],
                    y = df_arg.loc[:, Modules[i]],
                    name = Modules[i]+"-Response Data")
            )
            # Module Expected Revenue Values
            traces.append(
                go.Scatter(
                    x = df_arg.loc[:, 'Frequency'],
                    y = df_arg.loc[:, Modules[i]+'_REV'],
                    name = Modules[i]+"-Expected Rev",
                    yaxis= 'y2')
            )
            # Add the two Traces to the module dictionary to pull from
            Mod_dict.update({Modules[i]:traces})
            #Loop should end here
            
    print("Here's a peek at the available modules to view:\n{}".format(Modules))
    module_selected = input('Please enter the module desired:\n')
    while module_selected not in Modules:
            module_selected = input('Invalid Choice: Please enter the module desired. (Case-sensitive)\n')
    data = Mod_dict[module_selected]
    layout = go.Layout(
        title ='Gabor Granger',
        
        legend = dict(
            orientation='h',
            xanchor='center',
            x=0.5,
            y=-0.25
        ),

        xaxis=dict(
            title='Price'
    #         domain=[0,1]
        ),

        yaxis=dict(
            title='% of Responses willing to pay',
            anchor='x',
            range=[0,1]
        ),

        yaxis2=dict(
            title='Expected Revenue',
            anchor='x',
            overlaying='y',
            side='right',
            range=[0,GG_Final['Frequency'].max()]
        )
    )

    plotly.offline.iplot({
        'data':data, 
        'layout':layout})

In [71]:
interactive_GG(GG_Final)

Here's a peek at the available modules to view:
['CAD', '911', 'RMS/FBR', 'Mobile', 'JMS', 'Full Suite', 'Finance', 'Community Development', 'Utilities', 'Asset/Work Management']
Please enter the module desired:
CAD


### Charge Type Distribution

In [72]:
# Create the data tables for both PA and PSJ isolating charge type distribution for respondents
charge_types = ['License Fees','Maintenance','Professional Services']
PA_subscription_indicies = pd.Index([x for x in PA_only_indicies if 
                                     x in wkgdf[wkgdf['PA-Hosted']==0].index])

PA_charge_type_df = wkgdf.loc[PA_subscription_indicies,
                              wkgdf.columns[wkgdf.columns.str.contains('Q20')]
                             ].where(wkgdf.loc[PA_subscription_indicies,
                                               wkgdf.columns[wkgdf.columns.str.contains('Q20')]
                                              ]!="---"
                                    ).astype('float64')

In [73]:
# Creating full set of index points to shape data
PSJ_subscription_indicies = pd.Index([x for x in PSJ_only_indicies if 
                                      x in wkgdf[wkgdf['PSJ-Hosted']==0].index])

PSJ_subscription_full_suite_indicies = pd.Index([x for x in PSJ_only_indicies if 
                                                 x in wkgdf[wkgdf['PSJ-Hosted']==0].index and 
                                                 x in wkgdf[wkgdf['PSJ-FULLSUITE OWN']==1.0].index])

PSJ_subscription_nonfull_suite_indicies = pd.Index([x for x in PSJ_only_indicies if 
                                                 x in wkgdf[wkgdf['PSJ-Hosted']==0].index and 
                                                 x not in wkgdf[wkgdf['PSJ-FULLSUITE OWN']==1.0].index]
                                                  )

PSJ_charge_type_df = wkgdf.loc[PSJ_subscription_indicies,
                              wkgdf.columns[wkgdf.columns.str.contains('Q19')]
                             ].where(wkgdf.loc[PSJ_subscription_indicies,
                                               wkgdf.columns[wkgdf.columns.str.contains('Q19')]
                                              ]!="---"
                                    ).astype('float64')

In [74]:
pd.DataFrame(pd.DataFrame(PSJ_charge_type_df.loc[PSJ_subscription_nonfull_suite_indicies,
                                    PSJ_charge_type_df.columns[0]:PSJ_charge_type_df.columns[2]
                                   ].values,
             columns=charge_types
            ).dropna(axis=0,how='all')\
             .sort_values(by='License Fees')\
             .mode().iloc[0]).transpose()

,License Fees,Maintenance,Professional Services
0,25.0,25.0,50.0


In [ ]:
PSJ_charge_type_df.loc[PSJ_non_full_suite_indicies,:]

In [ ]:
new_attempt = []
for i in range(3):
    new_attempt.append(pd.DataFrame(list(zip(PSJ_charge_type_df.loc[PSJ_subscription_nonfull_suite_indicies,
                                                       PSJ_charge_type_df.columns[i]
                                                      ].value_counts().values,
                                PSJ_charge_type_df.loc[PSJ_subscription_nonfull_suite_indicies,
                                                       PSJ_charge_type_df.columns[i]
                                                      ].value_counts().index
                               )
                           ))
                      )

In [ ]:
from functools import reduce

In [ ]:
new_attempt

In [ ]:
reduce(lambda left,right: left.join(right,rsuffix='_x'),new_attempt).sum()

In [ ]:
display(PSJ_charge_type_df.loc[PSJ_subscription_nonfull_suite_indicies,
                                    PSJ_charge_type_df.columns[1]
                                   ].value_counts().iloc[:4].plot(kind='barh',
                                                                  stacked=True))

In [ ]:
sns.barplot(x=PSJ_charge_type_df.loc[PSJ_subscription_nonfull_suite_indicies,
                                    PSJ_charge_type_df.columns[2]
                                   ].value_counts().iloc[:4].index,
            y=PSJ_charge_type_df.loc[PSJ_subscription_nonfull_suite_indicies,
                                    PSJ_charge_type_df.columns[2]
                                   ].value_counts().iloc[:4].values
           )

In [ ]:
pd.DataFrame(PSJ_charge_type_df.loc[PSJ_subscription_nonfull_suite_indicies,
                                    PSJ_charge_type_df.columns[0]:PSJ_charge_type_df.columns[2]
                                   ].values,
             columns=charge_types
            ).dropna(axis=0,how='all')\
             .sort_values(by='License Fees',ascending=True).plot(kind='barh',
                                       stacked=True);

## Software Decision Making

Software Decision-Making Quicklinks:
1. [Biggest SW Purchase Challenges](#Biggest-SW-Purchase-Challenges)
2. [What can vendors do to speed up the SW Purchase process?](#Fast-Track-Options)
3. [What SW companies are you familiar with? (PA)](#Company-Familiarity---PA)
4. [Competitor Brand Assessment - PA](#Competitor-Brand-Assessment---PA)
5. [What SW companies are you familiar with? (PSJ)](#Company-Familiarity---PSJ)
6. [Competitor Brand Assessment - PSJ](#Competitor-Brand-Assessment---PSJ)
7. [CentralSquare Confidence Assessment](#CentralSquare-Confidence-Assessment)
8. [Most Influential Information Source](#Most-Influential-Information-Source)
9. [Media Consumption by Medium](#Media-Consumption-by-Medium)
10. Best Times to Call
    - [PA Times](#Heatmap---PA)
    - [PSJ Times](#Heatmap---PSJ)
11. [Travel & Conference Budget Changes](#Travel-&-Conference-Budget)
12. [Top Shows & Trade Show Attendence](#Top-Shows-&-Trade-Show-Attendence)

Go to [Next](#Products) or [Previous](#Pricing-Analysis)

Return to [Top](#Libraries)

### Heatmap of Time

#### Data

In [75]:
# Create the data tables for both PA and PSJ isolating best time to call availability for respondents
PA_time_avail_array = wkgdf.loc[PA_only_indicies,wkgdf.columns[wkgdf.columns.str.contains('Q28')]].values
PSJ_time_avail_array = wkgdf.loc[PSJ_only_indicies,wkgdf.columns[wkgdf.columns.str.contains('Q28')]].values

# Clean up into better data frames
time_avail_headers = ['Monday','Tuesday','Wednesday','Thursday','Friday']
PA_time_avail_df = pd.DataFrame(PA_time_avail_array,columns=time_avail_headers)
PSJ_time_avail_df = pd.DataFrame(PSJ_time_avail_array,columns=time_avail_headers)

In [76]:
# Isolate the unique values chosen for the survey
time_of_day_slots = np.unique(','.join(np.unique(PA_time_avail_df[~PA_time_avail_df.isin(['---'])].values.tolist()
                                                )
                                      ).split(',')
                             ).tolist()[:-1]

In [77]:
# Sorting order
weekday_order = {}
for i,x in enumerate(time_avail_headers):
    weekday_order.update({x:i})

inv_weekday_order = {v: k for k, v in weekday_order.items()}

time_of_day_order = {
    'Before 9am':0,
    '9-12noon':1,
    '12-3pm':2,
    '3-5pm':3,
    'After 5pm':4
}

inv_time_of_day_order = {v: k for k, v in time_of_day_order.items()}

In [78]:
def freq_call_table(data=None,time_slots=None):
    """
    
    """
    frequency_list = []
    for i,x in enumerate(time_slots):
        for j,y in enumerate(data.columns):
            frequency_list.append([y,x,data[y].isin([x]).sum()])
    freq_df = pd.DataFrame(frequency_list, columns=['Weekday','Time of Day','Frequency'])
    return freq_df

In [79]:
PA_best_time_to_call = freq_call_table(data=PA_time_avail_df,
                                       time_slots=time_of_day_slots)
PSJ_best_time_to_call = freq_call_table(data=PSJ_time_avail_df,
                                        time_slots=time_of_day_slots)

In [80]:
PA_BTC_pivot = PA_best_time_to_call.pivot('Time of Day','Weekday').reindex(index=list(time_of_day_order.keys()),
                                                                           columns=list(zip(
                                                                               ['Frequency']*len(time_avail_headers),
                                                                               time_avail_headers)
                                                                                       )
                                                                          )
PSJ_BTC_pivot = PSJ_best_time_to_call.pivot('Time of Day','Weekday').reindex(index=list(time_of_day_order.keys()),
                                                                           columns=list(zip(
                                                                               ['Frequency']*len(time_avail_headers),
                                                                               time_avail_headers)
                                                                                       )
                                                                          )

#### Heatmap - PA

In [81]:
generic_heatmap(PA_BTC_pivot,
                title='Best Time to Call - PA',
                N=len(PA_time_avail_df))

N = 106


#### Heatmap - PSJ

In [82]:
generic_heatmap(PSJ_BTC_pivot,
                title='Best Time to Call - PSJ',
                N=len(PSJ_time_avail_df))

N = 294


Products
-------------

1. [Placeholder]

Go to [Next](#Demployment-Method) or [Previous](#Software-Decision-Making)

Return to [Top](#Libraries)

Deployment Method
----------------------------

1. 

Go to [Next](#Payment-Method) or [Previous](#Products)

Return to [Top](#Libraries)

In [ ]:
# PA
wkgdf.loc[PA_only_indicies,wkgdf.columns[wkgdf.columns.str.contains('Q45')]]

In [ ]:
# PSJ
wkgdf.loc[PSJ_only_indicies,wkgdf.columns[wkgdf.columns.str.contains('Q45')]]

Payment Method
------------------------

1. 

Go to [Previous](#Products)

Return to [Top](#Libraries)